In [144]:


import numpy as np
from matplotlib import pyplot as plt

from qutrit_rpe import *

%load_ext autoreload 
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [145]:
xvec = random_error_vector(0.001, 0.01)
qids = ['Q0', 'Q1']
depol_rates = [0.001, 0.01] # 1 and 2 qubit depolarizing rates

In [146]:
parse_error_vector(xvec, qids)

{'single_qutrit': {'Q0': {'X01': -0.0068462911725404125,
   'phase01': 0.0312728881784852,
   'X12': 0.008077475558309618,
   'phase12': -0.00924342075296023},
  'Q1': {'X01': 0.018286347392094424,
   'phase01': 0.007553071830512371,
   'X12': -0.05432948425445634,
   'phase12': -0.0032713654550006674}},
 'two_qutrit': {'phi1': -0.04497121925534264,
  'phi2': 0.07570253166988766,
  'phi3': -0.03538352982894862,
  'phi4': -0.17936143052270742,
  'phi5': 0.013641631325305326,
  'phi6': -0.04873039506090418,
  'phi7': -0.008638990751778297,
  'phi8': 0.09204126250151211}}

In [147]:
target_model = make_two_qutrit_model(np.zeros(16), single_qutrit_depol=0.0, two_qutrit_depol=0.0, qids=qids)
noise_model = make_two_qutrit_model(xvec, single_qutrit_depol=depol_rates[0], two_qutrit_depol=depol_rates[1], qids=qids)

In [174]:
def gX01(qid):
    return [(f'Gx01', qid)]

def gY01(qid):
    return [(f'Gz01', qid)] + [(f'Gx01', qid)] + [(f'Gx01', qid)]*3

def gX12(qid):
    return [(f'Gx12', qid)]

def gY12(qid):
    return [(f'Gz12', qid)] + [(f'Gx12', qid)] + [(f'Gx12', qid)]*3

def gZ01(qid):
    return [(f'Gz01', qid)]

def gZ12(qid):
    return [(f'Gz12', qid)]

def gX01_inv(qid):
    return [(f'Gx01', qid)]*3 + [(f'Gz12', qid)]*4

def gX12_inv(qid):
    return [(f'Gx12', qid)]*3 + [(f'Gz01', qid)]*4

def gY01_inv(qid):
    return gY01(qid) + gY01(qid) + gY01(qid)

def gY12_inv(qid):
    return gY12(qid) + gY12(qid) + gY12(qid)

def check_inverse_defs():
    ux01 = modelX01(0,0)
    uz01 = modelZ01()
    ux12 = modelX12(0,0)
    uz12 = modelZ12()
    uy01 = uz01@uz01@uz01@ux01@uz01
    uy12 = uz12@uz12@uz12@ux12@uz12

    print(np.all(np.isclose(ux01@(ux01@ux01@ux01@uz12@uz12@uz12@uz12), -np.eye(3))))
    print(np.all(np.isclose(ux12@(ux12@ux12@ux12@uz01@uz01@uz01@uz01), -np.eye(3))))
    print(np.all(np.isclose(uy01@(uy01@uy01@uy01), np.eye(3))))
    print(np.all(np.isclose(uy12@(uy12@uy12@uy12), np.eye(3))))

check_inverse_defs()

True
True
True
True


In [199]:
from pygsti.circuits import Circuit

def make_rpe_circuit(germ, prep, meas, depth, line_labels):
    prep_circ = Circuit(prep, line_labels=line_labels)
    germ_circ = Circuit(germ, line_labels=line_labels)
    meas_circ = Circuit(meas, line_labels=line_labels)
    return prep_circ + germ_circ * depth + meas_circ

class RPEDesign1QT:
    def __init__(self, depths, qid, line_labels=None):
        self.depths = depths
        self.qid = qid
        if line_labels is None:
            line_labels = [qid]
        self.line_labels = line_labels
        self.circuit_dict = self._construct()
        self.circ_list = self._make_circuit_list()


    def _make_circuit_list(self):
        circs = []
        for param_label in self.circuit_dict.keys():
            for type_label in self.circuit_dict[param_label].keys():
                circs.extend(self.circuit_dict[param_label][type_label])
        return pygsti.remove_duplicates(circs)

    

    def _construct(self):
        circ_dict = {
            'Phase01': {},
            'Phase12': {},
            'X01 overrot': {},
            'X12 overrot': {}
        }
        circ_dict['X01 overrot']['I'] = self.make_x01_overrot_cos_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['X01 overrot']['Q'] = self.make_x01_overrot_sin_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['X12 overrot']['I'] = self.make_x12_overrot_cos_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['X12 overrot']['Q'] = self.make_x12_overrot_sin_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['Phase01']['I'] = self.make_phase01_cos_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['Phase01']['Q'] = self.make_phase01_sin_circuits(self.depths, self.qid, self.line_labels)
        return circ_dict

    def make_x01_overrot_cos_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX01(qid), [], [], depth, line_labels) for depth in depths]
    
    def make_x01_overrot_sin_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX01(qid), gX01(qid), [], depth, line_labels) for depth in depths]
    
    def make_x12_overrot_cos_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX12(qid), [], [], depth, line_labels) for depth in depths]
    
    def make_x12_overrot_sin_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX12(qid), gX12(qid), [], depth, line_labels) for depth in depths]
    
    def make_phase01_cos_circuits(self, depths, qid, line_labels):
        prep = gX01(qid) + gX01(qid) + gX12(qid) + gY01(qid) 
        meas = gY01_inv(qid) + gX12_inv(qid) + gX01(qid)*6
        return [make_rpe_circuit(gX01(qid), prep, meas, depth, line_labels) for depth in depths]

    def make_phase01_sin_circuits(self, depths, qid, line_labels):
        prep = gX01(qid) + gX01(qid) + gX12(qid) + gY01(qid) 
        meas = gY01_inv(qid) + gY12_inv(qid) + gX01(qid)*6
        return [make_rpe_circuit(gX01(qid), prep, meas, depth, line_labels) for depth in depths]
    


In [200]:
depths = [2**i for i in range(6)]
edesign = RPEDesign1QT(depths, qids[0], qids)
edesign.circuit_dict

{'Phase01': {'I': [Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx12:Q0Gx12:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0@(Q0,Q1)),
   Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx12:Q0Gx12:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0@(Q0,Q1)),
   Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx12:Q0Gx12:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0@(Q0,Q1)),
   Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01

In [190]:
print(edesign.circ_list[0])

Qubit Q0 ---|Gx01|---
Qubit Q1 ---|    |---



In [193]:
target_model.probabilities(edesign.circ_list[2])

OutcomeLabelDict([(('00',), 0.4999999999999994),
                  (('01',), -2.1684043449710089e-16),
                  (('02',), -1.1102230246251565e-16),
                  (('10',), 0.4999999999999999),
                  (('11',), -1.231653667943533e-16),
                  (('12',), -2.688821387764051e-16),
                  (('20',), -5.377642775528102e-17),
                  (('21',), -2.1684043449710089e-16),
                  (('22',), -1.942890293094024e-16)])